<a href="https://colab.research.google.com/github/galenzo17/AI-personal-test/blob/main/api_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 1. Instalar dependencias
!pip install fastapi uvicorn pydantic pyngrok scikit-learn nest_asyncio

# 2. Crear y guardar el modelo
import pickle
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

# Cargar datos de ejemplo
iris = load_iris()
X, y = iris.data, iris.target

# Dividir en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Entrenar un modelo simple
model = RandomForestClassifier()
model.fit(X_train, y_train)

# Guardar el modelo en un archivo .pkl
with open('model.pkl', 'wb') as f:
    pickle.dump(model, f)

print("Modelo entrenado y guardado como 'model.pkl'.")

# 3. Definir la API con FastAPI
from fastapi import FastAPI
from pydantic import BaseModel
import numpy as np

# Definir la estructura de los datos de entrada
class InputData(BaseModel):
    sepal_length: float
    sepal_width: float
    petal_length: float
    petal_width: float

# Inicializar la aplicación FastAPI
app = FastAPI()

# Cargar el modelo al inicio
with open('model.pkl', 'rb') as f:
    model = pickle.load(f)

# Definir el endpoint /predict
@app.post("/predict")
def predict(data: InputData):
    # Convertir los datos de entrada a un array numpy
    input_features = np.array([[data.sepal_length, data.sepal_width, data.petal_length, data.petal_width]])

    # Realizar la predicción
    prediction = model.predict(input_features)

    # Obtener el nombre de la especie
    species = iris.target_names[prediction[0]]

    return {"prediction": species}

# 4. Configurar y ejecutar el servidor con Uvicorn y ngrok
import nest_asyncio
from pyngrok import ngrok
import uvicorn
import threading

# Permitir que múltiples eventos asíncronos se ejecuten en Colab
nest_asyncio.apply()

# Configurar ngrok para exponer el puerto 8000
public_url = ngrok.connect(8000)
print(f"API está disponible en: {public_url}")

# Función para ejecutar Uvicorn
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Ejecutar Uvicorn en un hilo separado
thread = threading.Thread(target=run)
thread.start()

# 5. Probar la API
import time
import requests

# Esperar unos segundos para que el servidor arranque
time.sleep(5)

# Reemplaza con tu URL de ngrok
api_url = public_url + "/predict"

# Datos de ejemplo para la predicción
input_data = {
    "sepal_length": 5.1,
    "sepal_width": 3.5,
    "petal_length": 1.4,
    "petal_width": 0.2
}

# Realizar la petición POST
response = requests.post(api_url, json=input_data)

# Mostrar la respuesta
print(response.json())

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.7/94.7 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.7/63.7 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.3/73.3 kB 3.8 MB/s eta 0:00:00
